In [0]:
import pickle
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pyspark.sql import SparkSession
sc = spark.sparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, StringType, FloatType, ArrayType, DoubleType, StructType, StructField
sqlContext = SQLContext(sc)

In [0]:
base_save_path = "s3://mag-model-data/raw_mag_data/"
iteration_save_path = "s3://mag-model-data/V2/iteration_1/"

## Getting Level 2 Parents

In [0]:
journal_join_query = \
"""
SELECT e.paper_id, e.normalized_name as level_one, f.normalized_name as level_two
FROM (SELECT distinct a.paper_id, b.normalized_name
      FROM (SELECT paper_id, field_of_study as field_of_study_id
            FROM mag_advanced_paper_fields_of_study) a
      JOIN (SELECT field_of_study_id, normalized_name
            FROM mag_advanced_fields_of_study 
            WHERE level = 1) b
            ON a.field_of_study_id=b.field_of_study_id ) e
JOIN (SELECT distinct c.paper_id, d.normalized_name
      FROM (SELECT paper_id, field_of_study as field_of_study_id
            FROM mag_advanced_paper_fields_of_study) c
      JOIN (SELECT field_of_study_id, normalized_name
            FROM mag_advanced_fields_of_study 
            WHERE level = 2) d
            ON c.field_of_study_id=d.field_of_study_id) f
ON e.paper_id=f.paper_id
"""

In [0]:
all_data = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", journal_join_query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
all_data.printSchema()

root
-- paper_id: long (nullable = true)
-- level_one: string (nullable = true)
-- level_two: string (nullable = true)

In [0]:
all_data.orderBy(F.rand()).show(20)

+----------+--------------------+--------------------+
 paper_id| level_one| level_two|
+----------+--------------------+--------------------+
2882575706|condensed matter ...| superconductivity|
2109685119| multimedia| task|
3007144910| mechanics| transverse plane|
 110406168| veterinary medicine| healthy subjects|
2915435194|environmental health| health improvement|
2336168887| crystallography| crystallization|
2581259450| surgery| hyperplasia|
3025824940| cancer research| trabecular bone|
1976793708| classical mechanics| acceleration|
2725729045| topology| tooth number|
2848290644| composite material| mold|
1996566346|mathematical opti...|multi objective o...|
2384781223|electronic engine...| continuous signal|
3010744300|chemical engineering| thermal|
1995142175| nursing| interquartile range|
2399773826|environmental res...| service|
 418752685| neuroscience| eye movement|
1522751997| optics| skin pathology|
2823457946| food science| blood pressure|
2028181566| pediatrics| action|
+----------+--------------------+--------------------+
only showing top 20 rows

In [0]:
all_data.cache().count()

Out[14]: 893276865

In [0]:
w1 = Window.partitionBy('level_two').orderBy(F.col('weighted').desc())

In [0]:
# Getting all pairs of level ones and level twos across all papers in MAG
one_two_pair_counts = all_data.groupby(['level_two','level_one']).count() \
.join(all_data.groupby('level_one').count().select('level_one', F.col('count').alias('level_one_count')), on='level_one')

In [0]:
# Weighting counts of level ones for each level two by the total number of level ones. This is done
# to make sure that high-frequency level ones are not dominating
one_two_pair_counts \
.select('level_two','level_one','count','level_one_count', 
        (F.col('count')/F.col('level_one_count')).alias('weighted')) \
.withColumn('rank', F.row_number().over(w1)).filter(F.col('rank') <=15) \
.withColumn('topic_list', F.collect_list(F.col('level_one')).over(w1)) \
.groupby('level_two').agg(F.max(F.col('topic_list')).alias('topic_list')) \
.coalesce(1).write.mode('overwrite').parquet(f"{base_save_path}level_2_parents")

## Getting Level 3 Parents

In [0]:
journal_join_query = \
"""
SELECT e.paper_id, e.normalized_name as level_two, f.normalized_name as level_three
FROM (SELECT distinct a.paper_id, b.normalized_name
      FROM (SELECT paper_id, field_of_study as field_of_study_id
            FROM mag_advanced_paper_fields_of_study) a
      JOIN (SELECT field_of_study_id, normalized_name
            FROM mag_advanced_fields_of_study 
            WHERE level = 2) b
            ON a.field_of_study_id=b.field_of_study_id ) e
JOIN (SELECT distinct c.paper_id, d.normalized_name
      FROM (SELECT paper_id, field_of_study as field_of_study_id
            FROM mag_advanced_paper_fields_of_study) c
      JOIN (SELECT field_of_study_id, normalized_name
            FROM mag_advanced_fields_of_study 
            WHERE level = 3) d
            ON c.field_of_study_id=d.field_of_study_id) f
ON e.paper_id=f.paper_id
"""

In [0]:
all_data = spark.read \
.format("com.databricks.spark.redshift") \
.option("url", redshift_url) \
.option("user", "app_user") \
.option("password", redshift_password) \
.option("query", journal_join_query) \
.option("tempdir", base_save_path) \
.option("forward_spark_s3_credentials", True) \
.load()

In [0]:
all_data.printSchema()

root
-- paper_id: long (nullable = true)
-- level_two: string (nullable = true)
-- level_three: string (nullable = true)

In [0]:
all_data.show(20)

+----------+--------------------+-------------------+
 paper_id| level_two| level_three|
+----------+--------------------+-------------------+
2063220206| regeneration| actin cytoskeleton|
2063220206| mechanism| actin cytoskeleton|
2063220206| morphogenesis| actin cytoskeleton|
2398848774|psychological int...| acute care|
2398848774| amputation| acute care|
2398848774| complication| acute care|
 749747137| control system| slip ratio|
 749747137| torque| slip ratio|
 749747137| throttle| slip ratio|
 749747137| nonlinear system| slip ratio|
3123964653| cohort| gestation|
3123964653| population| gestation|
3123964653| cohort study| gestation|
3123964653| pregnancy| gestation|
2419318444| alpha|spondyloarthropathy|
2419318444|ankylosing spondy...|spondyloarthropathy|
2419318444| methotrexate|spondyloarthropathy|
2030452683| population| life expectancy|
2030452683| mosquito net| life expectancy|
2030452683| wet season| life expectancy|
+----------+--------------------+-------------------+
only showing top 20 rows

In [0]:
w1 = Window.partitionBy('level_three').orderBy(F.col('weighted').desc())

In [0]:
# Getting all pairs of level twos and level threes across all papers in MAG
two_three_pair_counts = all_data.groupby(['level_three','level_two']).count() \
.join(all_data.groupby('level_two').count().select('level_two', F.col('count').alias('level_two_count')), on='level_two')

In [0]:
# Weighting counts of level twos for each level three by the total number of level twos. This is done
# to make sure that high-frequency level twos are not dominating
two_three_pair_counts \
.select('level_three','level_two','count','level_two_count', 
        (F.col('count')/F.col('level_two_count')).alias('weighted')) \
.withColumn('rank', F.row_number().over(w1)).filter(F.col('rank') <=15) \
.withColumn('topic_list', F.collect_list(F.col('level_two')).over(w1)) \
.groupby('level_three').agg(F.max(F.col('topic_list')).alias('topic_list')) \
.coalesce(1).write.mode('overwrite').parquet(f"{base_save_path}level_3_parents")